In [18]:
from pydna.dseqrecord import Dseqrecord
from pydna.crispr import cas9, protospacer

guide = "GTTACTTTACCCGACGTCCCgttttagagctagaaatagcaagttaaaataagg"
target = "GTTACTTTACCCGACGTCCCaGG"

ex_guide = "GTTACTTTACCCGACGTCCC"
ex_target = "GTTACTTTACCCGACGTCCCaGG"

# Create an enzyme object with the guide RNA
enzyme = cas9("GTTACTTTACCCGACGTCCC")

print(enzyme.search("GTTACTTTACCCGACGTCCCaGG"))
print(len(ex_target))

dsqr_target = Dseqrecord(ex_target)
print(dsqr_target.seq.get_cutsites(enzyme))
print("REsulting cuts")
for i in dsqr_target.cut(enzyme):
    print(i.seq)


[18]
23
[((17, 0), cas9(GTT..CCC))]
REsulting cuts
GTTACTTTACCCGACGT
CCCaGG


In [2]:
"ATTTGTCAAACAGATTCCCA".lower()

'atttgtcaaacagattccca'

In [33]:
from pydna.dseqrecord import Dseqrecord
from pydna.crispr import cas9, protospacer

guide = "gttactttacccgacgtccc"

target = "GTTACTTTACCCGACGTCCCaGG"
rc_target = "CCtGGGACGTCGGGTAAAGTAAC"

# Create an enzyme object with the guide RNA
enzyme = cas9(guide.upper())

# Search for a cutsite in the target sequence
print(enzyme.search(target))
print(enzyme.search(rc_target))


[18]
[7]


# CRISPR Testing info

Plasmid addgene ID: 39295

RF: TTTCATATCTTCCTTTATATTCTATTAATTGAATTTCAAACATCGTTTTATTGAGCTCATTTACATCAACCGGTTCAATGtacccatacgatgttcctgactatgcgggctatccgtatgacgtcccggactatgcaggatcctatccatatgacgttccagattacgctCAAACAGTAATGATGGATGACATTCAAAGCACTGATTCTATTGCTGAAAAAGATAATCACTCTAATAATGAATCTAACTT

In [48]:
from pydna.dseqrecord import Dseqrecord
from pydna.parsers import parse as pydna_parse
import assembly2 as assembly
from pydna.crispr import cas9, _cas

RF = Dseqrecord("TTTCATATCTTCCTTTATATTCTATTAATTGAATTTCAAACATCGTTTTATTGAGCTCATTTACATCAACCGGTTCAATGtacccatacgatgttcctgactatgcgggctatccgtatgacgtcccggactatgcaggatcctatccatatgacgttccagattacgctCAAACAGTAATGATGGATGACATTCAAAGCACTGATTCTATTGCTGAAAAAGATAATCACTCTAATAATGAATCTAACTT")
guide = Dseqrecord("CATTACTGTTTGCATTGAAC")

file = open("/Users/dgruano/dev/learn/Genestorian/ShareYourCloning_backend/test_files/ase1.gb")
file_content = file.read()
target = pydna_parse(file_content)[0]

#target = Dseqrecord("ACGTgggAATT")
#RF = Dseqrecord("ACGTccAATT")

fragments = [target, RF]

# Use homologous_recombination as a reference
asm = assembly.Assembly(fragments, 80, use_all_fragments=True)
possible_assemblies = [a for a in asm.get_insertion_assemblies() if a[0][0] == 1]

# Extract target coordinates to check if the guide RNA is in the target
target_start = int(possible_assemblies[0][0][2].start)
target_end = int(possible_assemblies[0][1][3].end)

# Is the cutsite within the target? There needs to be at least 1 bp to the right and left of the cutsite

# Create an enzyme object with the guide RNA
guide = "CATTACTGTTTGCATTGAAC"
enzyme = cas9(guide.upper())

# cut_location = enzyme.search(target.seq)
# print(cut_location)

possible_cuts = [c for c in target.seq.get_cutsites(enzyme) if c[0][0] >= target_start and c[0][0] <= target_end]
print(possible_cuts)






[((851, 0), cas9(CAT..AAC))]


In [79]:
class test_cas9(cas9):
    pam = ".GG"
    size = 5
    fst5 = 2
    fst3 = -3

#guide = "TAAAA"
#target = Dseqrecord("ACGtaaaaAGGTGCA")
#RF = Dseqrecord("ACGcccGGT")
    
"""
EX0: Cut in the middle of HR seq
    ><
   taaaaPAM
   |||||
ACGtaaaaAGGTGCA
|||      |||
ACG......GGT
"""

guide = "GTaaa"
target = Dseqrecord("ACGtttGGTAAAAGGACGT")
RF = Dseqrecord("ACGcGGT")
"""
EX1: Cut on the right part of HR
        ><
       GTaaaPAM
       |||||
ACGtttGGTAAAAGGACGT
|||    |||
ACG....GTA
"""

#guide = "ACGtt"
#target = Dseqrecord("ACGttAGGCTGCA")
#RF = Dseqrecord("ACGccGGC")

"""
EX2: Cut on the left part of HR
 ><
ACGttPAM  
|||||
ACGttAGGCTGCA
 |||  |||
 CGT..GGC
"""

#guide = "GTaaa"
#target = Dseqrecord("ACGtttGGTAAAAGGACGT")
#RF = Dseqrecord("ACGcGGT")
"""
EX3: Cut to the right of HR
        ><
       GTaaaPAM
       |||||
ACGtttGGTAAAAGGACGT
|||   |||
ACG...GGT
"""

#guide = "GTaaa"
#target = Dseqrecord("TTTGGTaaaAGGACGT")
#RF = Dseqrecord("AAAccACG")

"""
EX4: Cut to the left of HR
     ><
    GTaaaPAM
    |||||PAM
TTTGGTaaaAGGACGT
      |||   |||
      AAA...ACG
"""

#guide = "GTaaa"
#target = Dseqrecord("ACGtttGGTAAAAGGTGCA")
#RF = Dseqrecord("ACGaaGGT")
"""
EX5: Double target HR
        ><
       GTaaaPAM
       |||||
ACGtttGGTAAAAGGTGCA
|||   |||
ACG...GGT
"""

fragments = [target, RF]
asm = assembly.Assembly(fragments, 3, use_all_fragments=True)
possible_assemblies = [a for a in asm.get_insertion_assemblies() if a[0][0] == 1]

print("Number of possible assemblies: ", len(possible_assemblies))
for a in possible_assemblies:
    print(assembly.assembly2str(a))

#print(assembly.assemble(fragments, possible_assemblies[0], False))

target_start = int(possible_assemblies[0][0][2].start)
target_end = int(possible_assemblies[0][1][3].end)

test_enzyme = test_cas9(guide.upper())
cut_pos = target.seq.get_cutsites(test_enzyme)

print()
print("Target start and end:")
print(target_start, target_end)
print()
print("Cut positions:")
print(cut_pos)

reparable_cuts = [c for c in cut_pos if c[0][0] > target_start and c[0][0] <= target_end]

print()
print("CRISPR-compatible cuts:")
print(possible_cuts)



Number of possible assemblies:  1
('1[0:3]:2[0:3]', '2[4:7]:1[6:9]')

Target start and end:
0 9

Cut positions:
[((9, 0), test_cas9(GTA..AAA))]

CRISPR-compatible cuts:
[((9, 0), test_cas9(GTA..AAA))]


In [41]:
print(target[840:870])

Dseqrecord
circular: False
size: 30
ID: part_NC_003424
Name: part_NC_003424
Description: Schizosaccharomyces pombe chromosome I, complete sequence
Database cross-references: BioProject:PRJNA127, BioSample:SAMEA3138176, Assembly:GCF_000002945.1
Number of features: 0
/molecule_type=DNA
/topology=linear
/data_file_division=CON
/date=21-MAR-2024
/keywords=['RefSeq', 'complete genome']
/source=Schizosaccharomyces pombe (fission yeast)
/organism=Schizosaccharomyces pombe
/taxonomy=['Eukaryota', 'Fungi', 'Dikarya', 'Ascomycota', 'Taphrinomycotina', 'Schizosaccharomycetes', 'Schizosaccharomycetales', 'Schizosaccharomycetaceae', 'Schizosaccharomyces']
/references=[Reference(title='The genome sequence of Schizosaccharomyces pombe', ...), Reference(title='Direct Submission', ...), Reference(title='Direct Submission', ...)]
/comment=        Annotation Provider :: PomBase
        Annotation Status   :: Full annotation
        Annotation Pipeline :: Eukaryotic Annotation Propagation Pipeline
       